In [2]:
from pprint import pprint
import pyodbc
import csv
import pandas as pd
import os
import glob

In [3]:
def connect_to_db():
    """Connects to the database."""

    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=109.156.30.146,1433;'
                          'Database=football_data2;'
                          'UID=footballdata;'
                          'PWD=tonyyeboah1;'
                          'Trusted_Connection=no;')

    cursor = conn.cursor()
    return cursor

In [4]:
def create_variant_query_filename(filename):
    """Creates a filename to store the results of the variant queries in."""
    with open(filename, 'w+') as f:
        headers = 'total_back_profit,back_roi,total_lay_profit,lay_roi,total_matches,best_odds,avg_odds,' \
                  'worst_odds'
        f.write(headers + '\n')
        f.close()

In [5]:
def create_query_filename(filename):
    """Creates a file to store the results of the queries in."""
    with open(filename, 'w+') as f:
        headers = 'Div,total_back_profit,back_roi,total_lay_profit,lay_roi,total_matches,overs_wins,' \
                  'overs_success_rate,best_odds,avg_odds,worst_odds'
        f.write(headers + '\n')
        f.close()

In [6]:
def prettify_output(cursor, results):
    """Prints the query output in a nice way on the console."""
    columns = cursor.description
    output = [{columns[index][0]: column for index, column in enumerate(value)} for value in results]
    pprint(output)

In [7]:
def variants(n, x):
    """Returns the stats from backing matches where X of the last N matches have had over 2.5 goals
    and stores the output in a .csv file."""
    if x > n:
        raise Exception('The number of matches with over 2.5 goals cannot exceed the number of total matches!')
    cursor = connect_to_db()
    cursor.execute(f'EXEC dbo.spVariants {n}, {x}')
    results = cursor.fetchall()
    prettify_output(cursor, results)

    # Write the results to .csv
    filename = f'{x} of {n}.csv'
    create_variant_query_filename(filename)
    with open(filename, 'a') as f:
        writer = csv.writer(f, delimiter=",", lineterminator="\n")
        writer.writerows(results)
        f.close()

    # Add a column for the variant
    results_db = pd.read_csv(filename)
    results_db['Variant'] = f'{x} of {n}'
    results_db.to_csv(filename)

In [8]:
def h2h_query(home, away):
    """Prints stats on a particular fixture."""
    cursor = connect_to_db()
    home = '"' + home + '"'  # Enclose in quotes so the query processes it correctly
    away = '"' + away + '"'
    cursor.execute(f'EXEC dbo.sph2h @HomeTeam = {home}, @AwayTeam = {away}')
    results = cursor.fetchall()
    prettify_output(cursor, results)

In [23]:
def top_league_teams_query(league_pos, div):
    """Returns stats betting on over 2.5 in matches where either team finished in the top X of the league
    in the previous season. If you want matches from all divisions, specify 'div' = NULL. If you want
    to see the stats from betting on over 2.5 in all games in a league, specify league_pos to be
    a large number (>24)"""
    cursor = connect_to_db()
    if div != 'NULL':
        div = '"' + div + '"'
    cursor.execute(f'EXEC dbo.TopXTeams @LeaguePos = {league_pos}, @Div = {div}')
    results = cursor.fetchall()
    prettify_output(cursor, results)
    
    if div == 'NULL':
        div_string = 'All'
    else:
        div_string = div.replace('"', '')

    filename = f'Top {league_pos} Teams in {div_string}.csv'
    create_query_filename(filename)
    with open(filename, 'a') as f:
        writer = csv.writer(f, delimiter=",", lineterminator="\n")
        writer.writerows(results)
        f.close()

    # Add a column for the variant
    results_db = pd.read_csv(filename)
    results_db['Top X'] = league_pos
    results_db.to_csv(filename)

In [10]:
def filter_data(div, home, away, goals, filename):
    """Filters the data however you like and saves the results in a .csv file. All parameters optional
    except the filename, specify = NULL if you want that parameter to be optional."""
    if goals % 0.25 != 0 or goals < 0:
        raise Exception('Invalid goals line provided!')
    cursor = connect_to_db()
    cursor.execute(f'EXEC filter_data @Div = {div}, @HomeTeam = {home}, @AwayTeam = {away}, @FTTG = {goals}'.replace \
                       ('"NULL"', 'NULL'))
    headers = 'Div,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,GBOvr,B365Ovr,BbMxOvr,BbAvOvr,POvr,\
              MaxOvr,AvgOvr,PCOvr,MaxCOvr,AvgCOvr,Season,FTTG,Fixture,BestOdds,BestCOdds,\
              MinOdds,MinCOdds,stake,lay_min_price,layC_min_price,lay_max_price,' \
              'layC_max_price,BestBackProfit,BestLayProfit,ID'
    results = cursor.fetchall()
    prettify_output(cursor, results)

    with open(filename, 'a') as f:
        f.write(headers + '\n')
        writer = csv.writer(f, delimiter=",", lineterminator="\n")
        writer.writerows(results)
        f.close()

In [11]:
def topx_teams_tg(div, overall_rank, home_rank, away_rank):
    """Returns the PnL and ROI stats from betting on over 2.5 in matches where either team's total goals ranking
    in the previous season (overall, home, away) is < X. Specify = NULL if you want the
    parameter to be optional"""
    cursor = connect_to_db()
    cursor.execute(f'EXEC spGoalsRanks @Div = {div}, @OverallGoalsRank = {overall_rank}, @HomeGoalsRank = {home_rank}, '
                   f'@AwayGoalsRank = {away_rank}'.replace('"NULL"', 'NULL'))
    results = cursor.fetchall()
    prettify_output(cursor, results)
    filename = f'{overall_rank}, {home_rank}, {away_rank}.csv'
    create_query_filename(filename)
    with open(filename, 'a') as f:
        writer = csv.writer(f, delimiter=",", lineterminator="\n")
        writer.writerows(results)
    res_db = pd.read_csv(filename)
    res_db['Combination'] = f'{overall_rank}, {home_rank}, {away_rank}'
    res_db.to_csv(filename)